In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import string
import keras
import nltk
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans

2024-01-02 05:33:39.911210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df=pd.read_csv('Edited_file.csv')

In [3]:
df.head()

,Text,Emotion
0,i know what happened might still feel real fee...,anger
1,when i heard about the way a parent of a frien...,anger
2,i is feeling particularly hostile shell say no...,anger
3,im feeling very bitter against knight in shini...,anger
4,i feel very disgusted by that i cant tolerated...,anger


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     30000 non-null  object
 1   Emotion  30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


In [5]:
df['Emotion'].value_counts()

Emotion
anger        5000
fear         5000
happiness    5000
joy          5000
neutral      5000
sadness      5000
Name: count, dtype: int64

In [6]:
df = df[df['Emotion'] != 'neutral']

In [7]:
emotion_to_mood = {
    'sadness': 'Sad',
    'joy': 'Excited',
    'anger': 'Angry',
    'fear': 'Fear',
    'happiness': 'Happy',
}
df['Mood'] = df['Emotion'].map(emotion_to_mood)

In [8]:
nlp = spacy.load('en_core_web_sm')
stopwords=list(STOP_WORDS)
punc=string.punctuation

In [9]:
def text_cleaner(sentence):
    doc=nlp(sentence)
    
    tokens=[]
    for token in doc:
        if token.lemma_!="-PRON-":
            temp=token.lemma_.lower().strip()
        else:
            temp=token.lower_
        tokens.append(temp)
        
    cleaned_tokens=[]
    for token in tokens:
        if token not in stopwords and token not in punc:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [10]:
x=df['Text']
y=df['Mood']

In [11]:
x=x.astype('U')

In [12]:
tfidf=TfidfVectorizer(tokenizer=text_cleaner)

In [13]:
RF=RandomForestClassifier()

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [15]:
model=Pipeline([('tfidf',tfidf),('RF',RF)])

In [16]:
model.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_cleaner at 0x13ab78040>)),
                ('RF', RandomForestClassifier())])

In [17]:
model.score(x_test,y_test)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


0.8298666666666666

In [18]:
y_pred=model.predict(x_test)

In [19]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       Angry       0.93      0.86      0.89      1473
     Excited       0.87      0.89      0.88      1497
        Fear       0.93      0.90      0.91      1533
       Happy       0.71      0.86      0.78      1510
         Sad       0.73      0.64      0.68      1487

    accuracy                           0.83      7500
   macro avg       0.83      0.83      0.83      7500
weighted avg       0.83      0.83      0.83      7500



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [20]:
from joblib import dump

In [27]:
model.predict(['''im angry'''])

array(['Angry'], dtype=object)

In [ ]:
# Assuming 'model' is your scikit-learn model
model_filename = 'RMA-EMO.joblib'
dump(model, model_filename)